# Verifying and Segmenting Sorties 

This code iterates through all of the tsv files and determines if there are segments of the data that are stopped or teleported based on the x, y, and z velocities (stopped) and both the positions and velocities of each direction. This code should create new 'segmented' files corresponding to each sortie.

The root directory should contain subdirectories of tsv files only.

The output should be a new directory (can be named using 'newdir' variable) with the sorties segmented with the 'good' sections - not teleported or stopped - separated out into new files named '(sortie number)seg(range of good values).min.tsv' (i.e., 12001477006seg1-1686.min.tsv).

In [21]:
import sys, os

rootdir = sys.argv[1] #navigate to folder with all tsv files
newdir = sys.argv[2] #new directory where new segmented files should appear

def stopped(data):
    threshold = 5 #These are good values to mess with!
    #checks x, then y, then z speeds
    if len(data) >= 7:
        if not type(data[5]) == str and not type(data[6]) == str and not type(data[7]) == str:
            if abs(float(data[5])) <= threshold and data[6] is not string and abs(float(data[6])) <= threshold and abs(float(data[7])) <= threshold:
                return True
            return False
    return False

def teleported(prevData, data):
    threshold = 100 #Another good value to change!
    if len(prevData) >= 7:
        if not type(prevData[4]) == str and not type(prevData[2]) == str and not type(prevData[3]) == str and not type(prevData[5]) == str and not type(prevData[6]) == str and not type(prevData[7]) == str:
            errorX = (float(prevData[2]) + (float(prevData[5]) * .2)) - float(data[2])
            errorY = (float(prevData[3]) + (float(prevData[6]) * .2)) - float(data[3])
            errorZ = (float(prevData[4]) + (float(prevData[7]) * .2)) - float(data[4])
            if errorX >= threshold or errorY >= threshold or errorZ >= threshold:
                return True
            return False
    return False

sorties = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        with open(subdir+"/"+file, "r") as f:
            #reads header
            f.readline()
            #Reads and throws out first line since they do not contain velocities
            f.readline()
            #reads second line
            prevData = f.readline().split('\t')
            #loops through the rest of the lines
            lines = f.readlines()
            inBad = False   #If we are in a bad section of sortie
            lastStep = 1     #The last time step we viewed (border timeStep)
            steps = []
            for line in lines:
                #determines if a line contains bad data
                badTime = False
                data = line.split('\t')
                badTime = stopped(data) or teleported(prevData, data)

                #Bad Time
                if badTime:
                    #Good -> bad
                    if not inBad:
                        inBad = True
                        #checks to see if the original data is bad
                        if not int(data[0]) - 1 == 2:
                            print(int(data[0]-1))
                            steps.append((True, lastStep, int(data[0])))
                            lastStep = int(data[0])
                #Good Time
                else:
                    #bad -> good
                    if inBad:
                        inBad = False
                        
                        steps.append((False, lastStep, int(data[0])))
                        lastStep = int(data[0])

                #updates PreviousData
                prevData = data

            #Finalize last step
            if not type(data[0]) == str:
                steps.append((not inBad, lastStep, int(data[0])))

        newSteps = []
        #check for good segments:
        for step in steps:
            if step[0] and (step[2] - step[1]) >= 1000:
                newSteps.append(step)

        if len(newSteps) >= 1:
            sorties.append([file, newSteps])

#attempt to make the new directory for the segmented sorties
if not (os.path.exists(newdir)):
    os.makedirs(newdir)

#gets the name of the sorties to split
sortieNum = [i[0] for i in sorties]

#separate the sorties
for subdir, dirs, files in os.walk(rootdir):
    for sortie in files:
        #Checks to see if sortie is in
        if sortie in sortieNum:
            #allows for easy reference to find split times
            sortieSplitIndex = sortieNum.index(sortie)

            sortieSplits = sorties[sortieSplitIndex][1]

            #open the file
            with open(subdir+"/"+sortie, "r") as bigFile:
                #Reads the header label
                bigFile.readline()
                #reads line without velocities
                bigFile.readline()
                #reads third line
                line = bigFile.readline()
                currLine = 3
                for split in sortieSplits:
                    startLine = split[1]
                    endLine = split[2]

                    with open(newdir+"/"+(sortie.split(".")[0])+"seg"+str(startLine)+"-"+str(endLine)+".min.tsv", "w+") as smallFile:
                        #continue copying till next timeSplit
                        while (currLine <= endLine):
                            smallFile.write(line)
                            line = bigFile.readline()
                            if not line == '':
                                currLine = currLine + 1
                            else:
                                break